імпорт необхідних бібліотек

In [120]:
# Імпорт бібліотек
import numpy as np
import pandas as pd
import nltk

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


первинна обробка даних: завантаження, виведення перших і останніх рядків, аналіз структури

> Додати блок цитати



In [121]:
# Завантаження датасету
emails = pd.read_csv('emails.csv')

# Виведення перших і останніх 5 рядків
print("Перші 5 рядків:")
print(emails.head())
print("\nОстанні 5 рядків:")
print(emails.tail())

# Інформація про датасет
print("\nІнформація про датасет:")
emails.info()

# Розмірність датасету
print(f"\nКількість рядків і стовпців: {emails.shape}")

Перші 5 рядків:
                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
2  Subject: unbelievable new homes made easy  im ...     1
3  Subject: 4 color printing special  request add...     1
4  Subject: do not have money , get software cds ...     1

Останні 5 рядків:
                                                   text  spam
5723  Subject: re : research and development charges...     0
5724  Subject: re : receipts from visit  jim ,  than...     0
5725  Subject: re : enron case study update  wow ! a...     0
5726  Subject: re : interest  david ,  please , call...     0
5727  Subject: news : aurora 5 . 2 update  aurora ve...     0

Інформація про датасет:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5728 non-null   object


Вивід показує перші 5 записів, кожен із яких має:

text: вміст повідомлення електронної пошти.
spam: мітку, яка вказує, чи є лист спамом (1) або не спамом (0).

Стовпець text має тип object (текст), а spam — int64 (ціле число)

перевіряємо чи є дублікати і видалимо їх

In [122]:
# Перевірка дублікатів
duplicates = emails.duplicated().sum()
print(f"\nКількість дублікатів: {duplicates}")

# Видалення дублікатів
emails = emails.drop_duplicates()
print(f"Новий розмір після видалення дублікатів: {emails.shape}")


Кількість дублікатів: 33
Новий розмір після видалення дублікатів: (5695, 2)


визначимо, де є відсутні дані у кожному стовпці

> Додати блок цитати



In [123]:
# Перевірка на пропущені значення
missing_values = emails.isnull().sum()
print("\nПропущені значення в кожному стовпці:")
print(missing_values[missing_values > 0])


Пропущені значення в кожному стовпці:
Series([], dtype: int64)


Проаналізуємо цільову змінну Prediction, щоб визначити співвідношення кількості записів про спам (spam) до не спаму (ham)

In [124]:
# Розподіл класів
class_distribution = emails['spam'].value_counts()
print("\nРозподіл класів (0 - не спам, 1 - спам):")
print(class_distribution)



Розподіл класів (0 - не спам, 1 - спам):
spam
0    4327
1    1368
Name: count, dtype: int64


9. Завантажити базу службових слів (from nltk.corpus import stopwords) / вивести перших 10 значень

Стоп-слова — слова, які не несуть значного сенсу (наприклад, "the", "is", "at"). видаляємо їх для покращення аналізу тексту.

In [125]:
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

# Завантажуємо стоп-слова
stop_words = stopwords.words('english')
print("\nПерші 10 стоп-слів:")
print(stop_words[:10])


Перші 10 стоп-слів:
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


10. Очищуємо з тексту:
- Пунктуацію
- Стоп-слова
- Перетворюємо текст у нижній регістр

In [126]:
def clean_text(text):
    text = text.lower() # переводимо текст до нижнього регістру
    text = text.translate(str.maketrans('', '', string.punctuation)) # Видаляємо пунктуацію
    # вдаляємо стоп-слова
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

# використовуємо функцію очищення до тексту
emails['cleaned_text'] = emails['text'].apply(clean_text)

# перші 5 очищених рядків
print("\nПерші 5 очищених текстів:")
print(emails['cleaned_text'].head())



Перші 5 очищених текстів:
0    subject naturally irresistible corporate ident...
1    subject stock trading gunslinger fanny merrill...
2    subject unbelievable new homes made easy im wa...
3    subject 4 color printing special request addit...
4    subject money get software cds software compat...
Name: cleaned_text, dtype: object


11. Токенізація тексту, вона розділяє текст на окремі слова (токени).

In [127]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Завантажуємо модуль токенізації
nltk.download('punkt_tab')

# Токенізація тексту
emails['tokens'] = emails['cleaned_text'].apply(word_tokenize)

# Виводимо токени для перших 5 рядків
print("\nТокени перших 5 рядків:")
print(emails['tokens'].head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



Токени перших 5 рядків:
0    [subject, naturally, irresistible, corporate, ...
1    [subject, stock, trading, gunslinger, fanny, m...
2    [subject, unbelievable, new, homes, made, easy...
3    [subject, 4, color, printing, special, request...
4    [subject, money, get, software, cds, software,...
Name: tokens, dtype: object


приклад, що відбулося:
"subject naturally irresistible corporate image"
->
['subject', 'naturally', 'irresistible', 'corporate', 'image']

12. Виконуємо векторизацію даних перетворивши текст у матрицю,
застосувавши метод TfidfVectorizer.  
13. Показуємо параметри набору після векторизації
14. Показ фрагменту матриці

Векторизація перетворює текстові дані в числовий формат, придатний для машинного навчання. У цьому випадку ми використовуємо TfidfVectorizer, який враховує важливість слів у текстах.

In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=3000)  # ініціалізуємо TfidfVectorizer і ставимо обмеження до 3000 найважливіших слів

X = tfidf_vectorizer.fit_transform(emails['cleaned_text']) # очищений текст перетворюємо на вектори

# мітки (spam/ham) перетворимо на окремий масив
y = emails['spam']
 #13
print("\nФорма матриці після векторизації:") #параметри матриці (кількість рядків/стовпців)
print(X.shape)  # (кількість зразків, кількість ознак)
 #14
# Показ фрагменту матриці
print("\nФрагмент матриці векторизованих даних:")
print(X.toarray()[:5])  # Перші 5 рядків матриці



Форма матриці після векторизації:
(5695, 3000)

Фрагмент матриці векторизованих даних:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


15.Розділити набір веторизованих даних та навчальну і тестову вибірки
у пропорції 80/20 і 70/30

In [129]:
from sklearn.model_selection import train_test_split

# Розділення у пропорції 80/20
X_train_80, X_test_20, y_train_80, y_test_20 = train_test_split(X, y, test_size=0.2, random_state=42)
print("\nРозміри навчальної та тестової вибірки (80/20):")
print(f"Навчальна вибірка: {X_train_80.shape}")
print(f"Тестова вибірка: {X_test_20.shape}")

# Розділення у пропорції 70/30
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=42)
print("\nРозміри навчальної та тестової вибірки (70/30):")
print(f"Навчальна вибірка: {X_train_70.shape}")
print(f"Тестова вибірка: {X_test_30.shape}")


Розміри навчальної та тестової вибірки (80/20):
Навчальна вибірка: (4556, 3000)
Тестова вибірка: (1139, 3000)

Розміри навчальної та тестової вибірки (70/30):
Навчальна вибірка: (3986, 3000)
Тестова вибірка: (1709, 3000)


16.Створюємо і навчаємо класифікатор для поділу emails на spam чи ham

In [130]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix


Навчання DecisionTreeClassifier |
вибірка 80/20

In [131]:
# набір гіперпараметрів для DecisionTreeClassifier
tree_params = [
    {"criterion": "gini", "max_depth": 10},
    {"criterion": "entropy", "max_depth": 15},
    {"criterion": "gini", "max_depth": 20}
]

print("\nРезультати для DecisionTreeClassifier:")
for params in tree_params:
    # ініціалізуємо класифікатор з різними гіперпараметрами
    tree_clf = DecisionTreeClassifier(**params, random_state=42)

    # Навчання на вибірці 80/20
    tree_clf.fit(X_train_80, y_train_80)

    # Прогноз на тестовій вибірці
    y_pred = tree_clf.predict(X_test_20)

    # Оцінка
    print(f"\nГіперпараметри: {params}")
    print(classification_report(y_test_20, y_pred))
    print("Матриця помилок:")
    print(confusion_matrix(y_test_20, y_pred))



Результати для DecisionTreeClassifier:

Гіперпараметри: {'criterion': 'gini', 'max_depth': 10}
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       843
           1       0.84      0.94      0.89       296

    accuracy                           0.94      1139
   macro avg       0.91      0.94      0.92      1139
weighted avg       0.94      0.94      0.94      1139

Матриця помилок:
[[792  51]
 [ 19 277]]

Гіперпараметри: {'criterion': 'entropy', 'max_depth': 15}
              precision    recall  f1-score   support

           0       0.98      0.93      0.96       843
           1       0.83      0.96      0.89       296

    accuracy                           0.94      1139
   macro avg       0.91      0.94      0.92      1139
weighted avg       0.94      0.94      0.94      1139

Матриця помилок:
[[785  58]
 [ 13 283]]

Гіперпараметри: {'criterion': 'gini', 'max_depth': 20}
              precision    recall  f1-score   support



Навчання AdaBoostClassifier | вибірка 80/20

In [132]:
# Гіперпараметри для AdaBoostClassifier
boost_params = [
    {"n_estimators": 50, "learning_rate": 1.0},
    {"n_estimators": 100, "learning_rate": 0.5},
    {"n_estimators": 150, "learning_rate": 0.1}
]

print("\nРезультати для AdaBoostClassifier:")
for params in boost_params:
    boost_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), **params, random_state=42) # Ініціалізація класифікатора з різними гіперпараметрами

    # Навчання на вибірці 80/20
    boost_clf.fit(X_train_80, y_train_80)

    # Прогноз на тестовій вибірці
    y_pred = boost_clf.predict(X_test_20)

    # оцінюємо
    print(f"\nГіперпараметри: {params}")
    print(classification_report(y_test_20, y_pred))
    print("Матриця помилок:")
    print(confusion_matrix(y_test_20, y_pred))



Результати для AdaBoostClassifier:


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Гіперпараметри: {'n_estimators': 50, 'learning_rate': 1.0}
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       843
           1       0.97      0.92      0.94       296

    accuracy                           0.97      1139
   macro avg       0.97      0.95      0.96      1139
weighted avg       0.97      0.97      0.97      1139

Матриця помилок:
[[834   9]
 [ 24 272]]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Гіперпараметри: {'n_estimators': 100, 'learning_rate': 0.5}
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       843
           1       0.94      0.94      0.94       296

    accuracy                           0.97      1139
   macro avg       0.96      0.96      0.96      1139
weighted avg       0.97      0.97      0.97      1139

Матриця помилок:
[[824  19]
 [ 17 279]]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Гіперпараметри: {'n_estimators': 150, 'learning_rate': 0.1}
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       843
           1       0.95      0.91      0.93       296

    accuracy                           0.96      1139
   macro avg       0.96      0.95      0.95      1139
weighted avg       0.96      0.96      0.96      1139

Матриця помилок:
[[830  13]
 [ 27 269]]


17.Виводимо прогноз і фактичні значення класифікаторів для навчального набору даних

DecisionTreeClassifier

In [133]:
print("\nПрогнози і фактичні значення для DecisionTreeClassifier (навчальна вибірка):")
# фактичні і передбачені значення
print(f"\nГіперпараметри: {params}")
print("Фактичні значення (перші 10):", y_train_80[:10].values)
print("Прогнозовані значення (перші 10):", y_train_pred[:10])


Прогнози і фактичні значення для DecisionTreeClassifier (навчальна вибірка):

Гіперпараметри: {'n_estimators': 150, 'learning_rate': 0.1}
Фактичні значення (перші 10): [0 1 0 0 0 0 1 1 0 0]
Прогнозовані значення (перші 10): [0 1 0 0 0 0 1 1 0 0]


AdaBoostClassifier

In [134]:
print("\nПрогнози і фактичні значення для AdaBoostClassifier (навчальна вибірка):")
# фактичні і передбачені значення
print(f"\nГіперпараметри: {params}")
print("Фактичні значення (перші 10):", y_train_80[:10].values)
print("Прогнозовані значення (перші 10):", y_train_pred[:10])


Прогнози і фактичні значення для AdaBoostClassifier (навчальна вибірка):

Гіперпараметри: {'n_estimators': 150, 'learning_rate': 0.1}
Фактичні значення (перші 10): [0 1 0 0 0 0 1 1 0 0]
Прогнозовані значення (перші 10): [0 1 0 0 0 0 1 1 0 0]



18. Оцінка роботи моделей на навчальному наборі даних за допомогою
матриці помилок




DecisionTreeClassifier

In [135]:
from sklearn.metrics import confusion_matrix

print("\nМатриця помилок для DecisionTreeClassifier (навчальна вибірка):")

# Матриця помилок
cm = confusion_matrix(y_train_80, y_train_pred)
print(f"\nГіперпараметри: {params}")
print("Матриця помилок:")
print(cm)



Матриця помилок для DecisionTreeClassifier (навчальна вибірка):

Гіперпараметри: {'n_estimators': 150, 'learning_rate': 0.1}
Матриця помилок:
[[3483    1]
 [   0 1072]]


AdaBoostClassifier

In [136]:
print("\nМатриця помилок для AdaBoostClassifier (навчальна вибірка):")

# Матриця помилок
cm = confusion_matrix(y_train_80, y_train_pred)
print(f"\nГіперпараметри: {params}")
print("Матриця помилок:")
print(cm)


Матриця помилок для AdaBoostClassifier (навчальна вибірка):

Гіперпараметри: {'n_estimators': 150, 'learning_rate': 0.1}
Матриця помилок:
[[3483    1]
 [   0 1072]]


- TN (True Negative) — правильно класифіковані не спами (ham),
- TP (True Positive) — правильно класифіковані спами.
- FP (False Positive) — не спам помилково класифікований як спам.
- FN (False Negative) — спам помилково класифікований як не спам.
- [[TN, FP],
- [FN, TP]]


19. потрібно вивести прогноз і фактичні значення для тестового набору даних, щоб оцінити, як добре моделі працюють на даних, які вони не бачили під час навчання.

DecisionTreeClassifier

In [137]:
print("\nПрогнози і фактичні значення для DecisionTreeClassifier (тестова вибірка):")

# Прогноз на тестовій вибірці
y_test_pred = tree_clf.predict(X_test_20)

print(f"\nГіперпараметри: {params}")
print("Фактичні значення (перші 10):", y_test_20[:10].values)
print("Прогнозовані значення (перші 10):", y_test_pred[:10])



Прогнози і фактичні значення для DecisionTreeClassifier (тестова вибірка):

Гіперпараметри: {'n_estimators': 150, 'learning_rate': 0.1}
Фактичні значення (перші 10): [0 1 0 0 0 1 0 0 0 0]
Прогнозовані значення (перші 10): [0 1 0 0 0 1 0 0 0 0]


AdaBoostClassifier

In [138]:
print("\nПрогнози і фактичні значення для AdaBoostClassifier (тестова вибірка):")

# Прогноз на тестовій вибірці
y_test_pred = boost_clf.predict(X_test_20)

print(f"\nГіперпараметри: {params}")
print("Фактичні значення (перші 10):", y_test_20[:10].values)
print("Прогнозовані значення (перші 10):", y_test_pred[:10])


Прогнози і фактичні значення для AdaBoostClassifier (тестова вибірка):

Гіперпараметри: {'n_estimators': 150, 'learning_rate': 0.1}
Фактичні значення (перші 10): [0 1 0 0 0 1 0 0 0 0]
Прогнозовані значення (перші 10): [0 1 0 0 0 1 0 0 0 0]


20. Оцінюємо роботу моделей на тестовому наборі даних за допомогою
матриці помилок.

- [[TN, FP],
- [FN, TP]]


- TN (True Negative): Правильно класифіковані не спами.
- FP (False Positive): Не спам помилково класифікований як спам.
- FN (False Negative): Спам помилково класифікований як не спам.
- TP (True Positive): Правильно класифіковані спами.

DecisionTreeClassifier

In [139]:
from sklearn.metrics import confusion_matrix
print("\nМатриця помилок для DecisionTreeClassifier (тестова вибірка):")
# Прогноз для тестової вибірки
y_test_pred = tree_clf.predict(X_test_20)
# Матриця помилок
cm = confusion_matrix(y_test_20, y_test_pred)
print("Матриця помилок:")
print(cm)



Матриця помилок для DecisionTreeClassifier (тестова вибірка):
Матриця помилок:
[[808  35]
 [ 24 272]]


AdaBoostClassifier

In [140]:
print("\nМатриця помилок для AdaBoostClassifier (тестова вибірка):")
# Прогноз для тестової вибірки
y_test_pred = boost_clf.predict(X_test_20)
# Матриця помилок
cm = confusion_matrix(y_test_20, y_test_pred)
print("Матриця помилок:")
print(cm)


Матриця помилок для AdaBoostClassifier (тестова вибірка):
Матриця помилок:
[[830  13]
 [ 27 269]]


21. Використовуємо метод SMOTE, який допомагає, коли дані для різних класів (як-от, spam і ham) сильно незбалансовані.

SMOTE створює нові синтетичні приклади для меншого класу (у нашому випадку spam).

приклад:
Було:
- ham: 900
- spam: 100
Після застосування SMOTE:
- ham: 900
- spam: 900 (SMOTE створив ще 800 прикладів спаму).

In [141]:
from imblearn.over_sampling import SMOTE
from collections import Counter

Балансування даних за допомогою SMOTE

In [142]:
# До застосування SMOTE
print("Розподіл класів до SMOTE:")
print(Counter(y_train_80))

# Ініціалізація SMOTE
smote = SMOTE(random_state=42)

# Застосування SMOTE до навчальної вибірки
X_train_80_smote, y_train_80_smote = smote.fit_resample(X_train_80, y_train_80)

# Після застосування SMOTE
print("\nРозподіл класів після SMOTE:")
print(Counter(y_train_80_smote))


Розподіл класів до SMOTE:
Counter({0: 3484, 1: 1072})

Розподіл класів після SMOTE:
Counter({0: 3484, 1: 3484})


Навчаємо моделей на збалансованих даних

DecisionTreeClassifier

In [143]:
print("\nРезультати для DecisionTreeClassifier після SMOTE:")

for params in tree_params:
    # навчаємо DecisionTreeClassifier на збалансованих даних
    tree_clf = DecisionTreeClassifier(**params, random_state=42)
    tree_clf.fit(X_train_80_smote, y_train_80_smote)

    # Прогноз на тестовій вибірці
    y_test_pred = tree_clf.predict(X_test_20)

    # Матриця помилок
    cm = confusion_matrix(y_test_20, y_test_pred)
    print(f"\nГіперпараметри: {params}")
    print("Матриця помилок:")
    print(cm)



Результати для DecisionTreeClassifier після SMOTE:

Гіперпараметри: {'criterion': 'gini', 'max_depth': 10}
Матриця помилок:
[[781  62]
 [ 10 286]]

Гіперпараметри: {'criterion': 'entropy', 'max_depth': 15}
Матриця помилок:
[[793  50]
 [ 12 284]]

Гіперпараметри: {'criterion': 'gini', 'max_depth': 20}
Матриця помилок:
[[812  31]
 [ 16 280]]


AdaBoostClassifier

In [144]:
print("\nРезультати для AdaBoostClassifier після SMOTE:")

for params in boost_params:
    # Навчання AdaBoostClassifier на збалансованих даних
    boost_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), **params, random_state=42)
    boost_clf.fit(X_train_80_smote, y_train_80_smote)

    # Прогноз на тестовій вибірці
    y_test_pred = boost_clf.predict(X_test_20)

    # Матриця помилок
    cm = confusion_matrix(y_test_20, y_test_pred)
    print(f"\nГіперпараметри: {params}")
    print("Матриця помилок:")
    print(cm)



Результати для AdaBoostClassifier після SMOTE:


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Гіперпараметри: {'n_estimators': 50, 'learning_rate': 1.0}
Матриця помилок:
[[823  20]
 [ 11 285]]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Гіперпараметри: {'n_estimators': 100, 'learning_rate': 0.5}
Матриця помилок:
[[821  22]
 [ 11 285]]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Гіперпараметри: {'n_estimators': 150, 'learning_rate': 0.1}
Матриця помилок:
[[828  15]
 [ 29 267]]


22. потрібно дослідити, як гіперпараметри впливають на точність класифікації спаму. Для цього ми обираємо щонайменше три комбінації гіперпараметрів для кожного класифікатора (DecisionTreeClassifier і AdaBoostClassifier) і аналізуємо їх вплив.

Аналіз для DecisionTreeClassifier

In [145]:
from sklearn.metrics import accuracy_score

print("\nДослідження гіперпараметрів для DecisionTreeClassifier:")

for params in tree_params:
    # Навчання моделі на збалансованих даних (після SMOTE)
    tree_clf = DecisionTreeClassifier(**params, random_state=42)
    tree_clf.fit(X_train_80_smote, y_train_80_smote)

    # Прогноз на тестовій вибірці
    y_test_pred = tree_clf.predict(X_test_20)

    # Оцінка точності
    accuracy = accuracy_score(y_test_20, y_test_pred)
    print(f"\nГіперпараметри: {params}")
    print(f"Точність: {accuracy:.4f}")



Дослідження гіперпараметрів для DecisionTreeClassifier:

Гіперпараметри: {'criterion': 'gini', 'max_depth': 10}
Точність: 0.9368

Гіперпараметри: {'criterion': 'entropy', 'max_depth': 15}
Точність: 0.9456

Гіперпараметри: {'criterion': 'gini', 'max_depth': 20}
Точність: 0.9587


Аналіз для AdaBoostClassifier

In [146]:
print("\nДослідження гіперпараметрів для AdaBoostClassifier:")

for params in boost_params:
    # Навчання моделі на збалансованих даних (після SMOTE)
    boost_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), **params, random_state=42)
    boost_clf.fit(X_train_80_smote, y_train_80_smote)

    # Прогноз на тестовій вибірці
    y_test_pred = boost_clf.predict(X_test_20)

    # Оцінка точності
    accuracy = accuracy_score(y_test_20, y_test_pred)
    print(f"\nГіперпараметри: {params}")
    print(f"Точність: {accuracy:.4f}")


Дослідження гіперпараметрів для AdaBoostClassifier:


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Гіперпараметри: {'n_estimators': 50, 'learning_rate': 1.0}
Точність: 0.9728


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Гіперпараметри: {'n_estimators': 100, 'learning_rate': 0.5}
Точність: 0.9710


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Гіперпараметри: {'n_estimators': 150, 'learning_rate': 0.1}
Точність: 0.9614
